In [149]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import sklearn
import math as m
from sklearn import *
# ignore warnings
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

### Load the dataset from kaggle <br/><br/> Link: https://www.kaggle.com/c/titanic/data

In [150]:
df_test=pd.read_csv(r"..\Data\Titanic_test.csv")


In [151]:
df_train=pd.read_csv(r"..\Data\Titanic_train.csv")

In [152]:
titanic = df_train.append(df_test, ignore_index=True)

In [153]:
df_test.shape,df_train.shape, titanic.shape

((418, 11), (891, 12), (1309, 12))

In [154]:
# create indexes to separate data later on
train_idx = len(df_train)
test_idx = len(titanic) - len(df_test)

In [155]:
titanic.head()

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1.0,PC 17599
2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,1.0,STON/O2. 3101282
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,1.0,113803
4,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,0.0,373450


In [156]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
Age            1046 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
Fare           1308 non-null float64
Name           1309 non-null object
Parch          1309 non-null int64
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Sex            1309 non-null object
SibSp          1309 non-null int64
Survived       891 non-null float64
Ticket         1309 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 122.8+ KB


In [157]:
# PassengerId can be removed from data for now
del titanic['PassengerId']

### Feature Engineering

In [158]:
#adding newfeature
titanic['title'] = titanic.Name.apply(lambda name: name.split(',')[1].split('.')[0].strip())
titanic.head()

,Age,Cabin,Embarked,Fare,Name,Parch,Pclass,Sex,SibSp,Survived,Ticket,title
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,3,male,1,0.0,A/5 21171,Mr
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,1,female,1,1.0,PC 17599,Mrs
2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,female,0,1.0,STON/O2. 3101282,Miss
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,1,female,1,1.0,113803,Mrs
4,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,3,male,0,0.0,373450,Mr


In [161]:
#no. of unique titles
print("There are " +str(titanic.title.nunique())+" unique titles.")
# show unique titles
print("\n", titanic.title.unique())

There are 18 unique titles.

 ['Mr' 'Mrs' 'Miss' 'Master' 'Don' 'Rev' 'Dr' 'Mme' 'Ms' 'Major' 'Lady'
 'Sir' 'Mlle' 'Col' 'Capt' 'the Countess' 'Jonkheer' 'Dona']


###### Normalize the titles

In [162]:
normalized_titles = {
    "Capt":       "Officer",
    "Col":        "Officer",
    "Major":      "Officer",
    "Jonkheer":   "Royalty",
    "Don":        "Royalty",
    "Sir" :       "Royalty",
    "Dr":         "Officer",
    "Rev":        "Officer",
    "the Countess":"Royalty",
    "Dona":       "Royalty",
    "Mme":        "Person",
    "Mlle":       "Person",
    "Ms":         "Person",
    "Mr" :        "Person",
    "Mrs" :       "Person",
    "Miss" :      "Person",
    "Master" :    "Royalty",
    "Lady" :      "Royalty"
}

In [164]:
titanic.title = titanic.title.map(normalized_titles)
print(titanic.title.value_counts())

Person     1219
Royalty      67
Officer      23
Name: title, dtype: int64


In [179]:
agg=titanic.groupby(['Sex','Pclass', 'title'])
pd.DataFrame(agg.Age.median())


Age
Sex    Pclass title        
female 1      Officer  49.0
              Person   35.0
              Royalty  39.0
       2      Person   28.0
       3      Person   22.0
male   1      Officer  52.0
              Person   41.5
              Royalty  12.0
       2      Officer  41.5
              Person   30.0
              Royalty   2.0
       3      Person   26.0
              Royalty   6.0

In [173]:
i=df_train.Age.isnull()
i.head()

0    False
1    False
2    False
3    False
4    False
Name: Age, dtype: bool

In [181]:
titanic.Age = agg.Age.apply(lambda x: x.fillna(x.median()))
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
Age         1309 non-null float64
Cabin       295 non-null object
Embarked    1307 non-null object
Fare        1308 non-null float64
Name        1309 non-null object
Parch       1309 non-null int64
Pclass      1309 non-null int64
Sex         1309 non-null object
SibSp       1309 non-null int64
Survived    891 non-null float64
Ticket      1309 non-null object
title       1309 non-null object
dtypes: float64(3), int64(3), object(6)
memory usage: 122.8+ KB


In [182]:
t2=titanic.isnull().sum()
t2

Age            0
Cabin       1014
Embarked       2
Fare           1
Name           0
Parch          0
Pclass         0
Sex            0
SibSp          0
Survived     418
Ticket         0
title          0
dtype: int64

In [183]:
titanic.Cabin = titanic.Cabin.fillna('Unknown')

In [185]:
frq = titanic.Embarked.value_counts()
frq

S    914
C    270
Q    123
Name: Embarked, dtype: int64

In [190]:
titanic.Embarked = titanic.Embarked.fillna(frq[0])
titanic.Fare=titanic.Fare.fillna(titanic.Fare.median())

In [192]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
Age         1309 non-null float64
Cabin       1309 non-null object
Embarked    1309 non-null object
Fare        1309 non-null float64
Name        1309 non-null object
Parch       1309 non-null int64
Pclass      1309 non-null int64
Sex         1309 non-null object
SibSp       1309 non-null int64
Survived    891 non-null float64
Ticket      1309 non-null object
title       1309 non-null object
dtypes: float64(3), int64(3), object(6)
memory usage: 122.8+ KB


In [196]:
titanic.Survived.value_counts(normalize=True)

0.0    0.616162
1.0    0.383838
Name: Survived, dtype: float64

In [197]:
titanic.groupby("Sex").Survived.mean()

Sex
female    0.742038
male      0.188908
Name: Survived, dtype: float64

###### Female Survived More than Male 

In [198]:
titanic.groupby(["Sex","Pclass"]).Survived.mean()

Sex     Pclass
female  1         0.968085
        2         0.921053
        3         0.500000
male    1         0.368852
        2         0.157407
        3         0.135447
Name: Survived, dtype: float64

###### class 1 survived more

In [199]:
titanic.describe()

,Age,Fare,Parch,Pclass,SibSp,Survived
count,1309.000000,1309.000000,1309.000000,1309.000000,1309.000000,891.000000
mean,29.243063,33.281086,0.385027,2.294882,0.498854,0.383838
std,13.306089,51.741500,0.865560,0.837836,1.041658,0.486592
min,0.170000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,22.000000,7.895800,0.000000,2.000000,0.000000,0.000000
50%,26.000000,14.454200,0.000000,3.000000,0.000000,0.000000
75%,36.000000,31.275000,0.000000,3.000000,1.000000,1.000000
max,80.000000,512.329200,9.000000,3.000000,8.000000,1.000000


In [200]:
titanic['familysize'] = titanic.Parch + titanic.SibSp + 1

In [206]:
titanic['Cabin']=titanic.Cabin.map(lambda first: first[0])
titanic.Cabin.value_counts(normalize=True)

U    0.774637
C    0.071811
B    0.049656
D    0.035141
E    0.031322
A    0.016807
F    0.016043
G    0.003820
T    0.000764
Name: Cabin, dtype: float64

In [286]:
titanic.Sex = titanic.Sex.map({"male": 0, "female":1})

In [287]:
pclass_dummies = pd.get_dummies(titanic.Pclass, prefix="Pclass")
title_dummies = pd.get_dummies(titanic.title, prefix="Title")
cabin_dummies = pd.get_dummies(titanic.Cabin, prefix="Cabin")
embarked_dummies = pd.get_dummies(titanic.Embarked, prefix="Embarked")

In [289]:
titanic_dummies = pd.concat([titanic, pclass_dummies, title_dummies, cabin_dummies, embarked_dummies], axis=1)

# drop categorical fields
titanic_dummies.drop(['Pclass', 'title', 'Cabin', 'Embarked'], axis=1, inplace=True)

titanic_dummies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 26 columns):
Age              1309 non-null float64
Fare             1309 non-null float64
Parch            1309 non-null int64
Sex              1309 non-null int64
SibSp            1309 non-null int64
Survived         891 non-null float64
familysize       1309 non-null int64
Pclass_1         1309 non-null uint8
Pclass_2         1309 non-null uint8
Pclass_3         1309 non-null uint8
Title_Officer    1309 non-null uint8
Title_Person     1309 non-null uint8
Title_Royalty    1309 non-null uint8
Cabin_A          1309 non-null uint8
Cabin_B          1309 non-null uint8
Cabin_C          1309 non-null uint8
Cabin_D          1309 non-null uint8
Cabin_E          1309 non-null uint8
Cabin_F          1309 non-null uint8
Cabin_G          1309 non-null uint8
Cabin_T          1309 non-null uint8
Cabin_U          1309 non-null uint8
Embarked_914     1309 non-null uint8
Embarked_C       1309 non-null uint8


In [290]:
df_train=titanic_dummies[:train_idx]
df_test=titanic_dummies[test_idx:]

In [291]:
print(df_train.shape)
df_train.head()

(891, 26)


,Age,Fare,Parch,Sex,SibSp,Survived,familysize,Pclass_1,Pclass_2,Pclass_3,...,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_U,Embarked_914,Embarked_C,Embarked_Q,Embarked_S
0,22.0,7.2500,0,0,1,0.0,2,0,0,1,...,0,0,0,0,0,1,0,0,0,1
1,38.0,71.2833,0,1,1,1.0,2,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,26.0,7.9250,0,1,0,1.0,1,0,0,1,...,0,0,0,0,0,1,0,0,0,1
3,35.0,53.1000,0,1,1,1.0,2,1,0,0,...,0,0,0,0,0,0,0,0,0,1
4,35.0,8.0500,0,0,0,0.0,1,0,0,1,...,0,0,0,0,0,1,0,0,0,1


In [292]:
print(df_test.shape)
df_test.head()

(418, 26)


,Age,Fare,Parch,Sex,SibSp,Survived,familysize,Pclass_1,Pclass_2,Pclass_3,...,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_U,Embarked_914,Embarked_C,Embarked_Q,Embarked_S
891,34.5,7.8292,0,0,0,NaN,1,0,0,1,...,0,0,0,0,0,1,0,0,1,0
892,47.0,7.0000,0,1,1,NaN,2,0,0,1,...,0,0,0,0,0,1,0,0,0,1
893,62.0,9.6875,0,0,0,NaN,1,0,1,0,...,0,0,0,0,0,1,0,0,1,0
894,27.0,8.6625,0,0,0,NaN,1,0,0,1,...,0,0,0,0,0,1,0,0,0,1
895,22.0,12.2875,1,1,1,NaN,3,0,0,1,...,0,0,0,0,0,1,0,0,0,1


##### Dividing finally into test and train for model to Train

In [293]:
print(df_train.Survived.dtype)
df_train.Survived.astype("int64" ,inplace=True).head()

float64


0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [297]:
X = df_train.drop('Survived', axis=1)
y=df_train["Survived"]
X.head()

,Age,Fare,Parch,Sex,SibSp,familysize,Pclass_1,Pclass_2,Pclass_3,Title_Officer,...,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_U,Embarked_914,Embarked_C,Embarked_Q,Embarked_S
0,22.0,7.2500,0,0,1,2,0,0,1,0,...,0,0,0,0,0,1,0,0,0,1
1,38.0,71.2833,0,1,1,2,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,26.0,7.9250,0,1,0,1,0,0,1,0,...,0,0,0,0,0,1,0,0,0,1
3,35.0,53.1000,0,1,1,2,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,35.0,8.0500,0,0,0,1,0,0,1,0,...,0,0,0,0,0,1,0,0,0,1


###### Training Data

In [298]:

X_test = df_test.drop('Survived', axis=1)
X_test.head()

,Age,Fare,Parch,Sex,SibSp,familysize,Pclass_1,Pclass_2,Pclass_3,Title_Officer,...,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_U,Embarked_914,Embarked_C,Embarked_Q,Embarked_S
891,34.5,7.8292,0,0,0,1,0,0,1,0,...,0,0,0,0,0,1,0,0,1,0
892,47.0,7.0000,0,1,1,2,0,0,1,0,...,0,0,0,0,0,1,0,0,0,1
893,62.0,9.6875,0,0,0,1,0,1,0,0,...,0,0,0,0,0,1,0,0,1,0
894,27.0,8.6625,0,0,0,1,0,0,1,0,...,0,0,0,0,0,1,0,0,0,1
895,22.0,12.2875,1,1,1,3,0,0,1,0,...,0,0,0,0,0,1,0,0,0,1


In [300]:
# create param grid object
log_params = dict(
    C = np.logspace(-5, 8, 15),
    penalty = ['l1', 'l2']
)
# instantiate logistic regressor
log = linear_model.LogisticRegression()

# load param grid and log model into GridSearcCV
logreg_cv = model_selection.GridSearchCV(estimator=log, param_grid=log_params, cv=5)

# fit model
logreg_cv.fit(X, y)
print("Tuned Logistic Regression Parameters: {}".format(logreg_cv.best_params_)) 
print("Best score is {}".format(logreg_cv.best_score_))

Tuned Logistic Regression Parameters: {'C': 0.4393970560760795, 'penalty': 'l2'}
Best score is 0.8282828282828283


###### Testing Data

In [299]:
%%time
pipe = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree=2, include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.SGDClassifier(loss="log",
                                       penalty = "elasticnet",
                                       learning_rate = "invscaling",
                                       eta0 = 0.01, 
                                       max_iter = 2000,
                                       tol = 1e-4
                                      ))
])


pipe.fit(X,y)


y_test_prob = pipe.predict_proba(X_test)
y_train_pred = pipe.predict(X)
y_test_pred = pipe.predict(X_test)

#plot_confusion_matrix(metrics.confusion_matrix(y, y_train_pred))

print("training r2:", metrics.r2_score(y, y_train_pred))

training r2: 0.3925798101811906
Wall time: 385 ms


### SGD Classifier

In [303]:
%%time

param_grid = {
    "poly__degree": np.linspace(1,2,3,dtype="int32"),
    "est__l1_ratio": np.linspace(0, 1, 10),
    "est__alpha": np.linspace(0.04, 0.09, 10)
}

gsearch = model_selection.GridSearchCV(cv=5, 
                                       estimator=pipe, 
                                       n_jobs=4, 
                                       param_grid=param_grid)
gsearch.fit(X, y)


Wall time: 1min 1s


In [157]:
est=gsearch.best_estimator_
print(est)
print(gsearch.best_score_)

Pipeline(memory=None,
     steps=[('poly', PolynomialFeatures(degree=4, include_bias=False, interaction_only=False)), ('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('est', SGDClassifier(alpha=0.04, average=False, class_weight=None, epsilon=0.1,
       eta0=0.01, fit_intercept=True, l1_ratio=0.0,
      ...et', power_t=0.5, random_state=None,
       shuffle=True, tol=0.0001, verbose=0, warm_start=False))])
0.8204264870931538


In [304]:
y_test_predict=pipe.predict(X_test)

In [308]:
y_test_predict[:10]
y_test_predict.astype('int32')

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,

In [311]:
temp=pd.read_csv(r"..\Data\Titanic_test.csv")

In [312]:
res=pd.DataFrame({"PassengerId": temp.PassengerId,"Survived ":y_test_pred})

In [313]:
res.to_csv(r"result.csv",index=False)


### RANDOM FOREST

In [356]:
%%time
pipe = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree=4, include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", ensemble.RandomForestClassifier(max_features=0.5, n_estimators= 10, 
        max_depth=8, criterion="entropy",min_samples_split=3, min_samples_leaf=7, random_state=1,n_jobs=4))
])

pipe.fit(X, y)
print("training: ", pipe.score(X, y))

training:  0.8978675645342312
Wall time: 4.88 s


In [366]:
%%time

param_grid ={
    "poly__degree": np.linspace(1,4,4,dtype="int32"),
    "est__max_depth": np.linspace(1,6,6,dtype="int32"),
    "est__min_samples_split" : np.linspace(2,4,3,dtype="int32"),
    "est__min_samples_leaf" : np.linspace(2,5,4,dtype="int32"),
}

gsearch = model_selection.GridSearchCV(cv=10, 
                                       estimator=pipe, 
                                       n_jobs=4, 
                                       param_grid=param_grid)
gsearch.fit(X, y)


Wall time: 41min 37s


In [367]:
gsearch

GridSearchCV(cv=10, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('poly', PolynomialFeatures(degree=4, include_bias=False, interaction_only=False)), ('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('est', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=8, max_features=0.5, max_leaf_...estimators=10, n_jobs=4,
            oob_score=False, random_state=1, verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'poly__degree': array([1, 2, 3, 4]), 'est__max_depth': array([1, 2, 3, 4, 5, 6]), 'est__min_samples_split': array([2, 3, 4]), 'est__min_samples_leaf': array([2, 3, 4, 5])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [369]:
gsearch.best_score_ , gsearch.best_params_, gsearch.best_estimator_

(0.8383838383838383,
 {'est__max_depth': 5,
  'est__min_samples_leaf': 5,
  'est__min_samples_split': 2,
  'poly__degree': 2},
 Pipeline(memory=None,
      steps=[('poly', PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)), ('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('est', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
             max_depth=5, max_features=0.5, max_leaf_...estimators=10, n_jobs=4,
             oob_score=False, random_state=1, verbose=0, warm_start=False))]))

In [370]:
est=gsearch.best_estimator_

In [371]:
y_test_pred = est.predict(X_test)
y_test_predict.astype('int32')

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [374]:
y_test_predict=pipe.predict(X_test)
y_test_predict.astype('int32')

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [375]:
resRF=pd.DataFrame({"PassengerId": temp.PassengerId,"Survived ":y_test_pred})

In [377]:
resRF.to_csv("res.csv",index=False)

#### Upload the "result.csv" to kaggle <br/><br/> -------------------------------------------------------------------------------------------END-------------------------------------------------------------------------------------------------